In [1]:
    import IPython, toolz.curried as toolz, base64, vdom

In [ ]:
    mimes = toolz.pipe("""
        50 text/html
        70 text/latex
        80 image/svg+xml
        90 image/bmp image/png image/jpeg image/gif'
        60 text/markdown
        110 text/javascript application/javascript 
        120 text/plain
        """, str.strip, str.splitlines, toolz.map(toolz.compose(toolz.juxt(toolz.compose(int, next), list), iter, str.split)), list)

In [ ]:
    image_types = list(map('image/'.__add__, ('bmp', 'jpeg', 'gif', 'png')))

In [ ]:
    def flatten(str): return ''.join(str.splitlines())

In [ ]:
    import CommonMark

In [ ]:
    md = lambda str: CommonMark.render.html.HtmlRenderer().render(CommonMark.Parser().parse(str))

In [ ]:
    import vdom, base64, IPython

    def flatten(string):
        return ''.join(map(str.strip, string.splitlines()))

    def format_images(type, bundle):
        str = bundle[type]
        
        
        if isinstance(str, bytes):
            str = base64.b64encode(str).decode('utf-8')
            
        if type in ('image/svg+xml', 'text/html'):
            return IPython.display.HTML(flatten(str))
            
        if str.startswith('http'):
            return vdom.img(src=str)

        return vdom.img(src=f"data:{type};base64,{str}")

In [ ]:
  def ipython_formatter(object, html=False):
        import mistune
        format = IPython.get_ipython().display_formatter.format
        bundle, metadata = format(object)
        for type in toolz.pipe(mimes, toolz.map(toolz.second), toolz.concat):
            if type == 'text/plain': return object
            if type in bundle: 
                object = bundle[type]
                if type.startswith('image'):
                    bundle.update(format(format_images(type, bundle))[0])
                if html and type == 'text/markdown':
                    bundle.update({'text/markdown': md(object)})
                return bundle[type]

In [ ]:
    environment = __import__('jinja2').Environment(enable_async=False, finalize=ipython_formatter)
    environment.globals.update(vars(__import__('builtins')))

In [ ]:
    import string, collections

In [8]:
    class IPythonFormatDict(collections.UserDict):
        def __getitem__(self, object):
            return ipython_formatter(super().__getitem__(object))

In [9]:
    class IPythonDollarTemplate(string.Template):
        def safe_substitute(self, *args, **kw):
            return super().safe_substitute(IPythonFormatDict(collections.ChainMap(kw, *args)))
        
    